# B — PML tuning (project notebook)

This notebook is **project-native**: it uses your `core/` and `operators/` modules.

It:
- fixes the physical domain to **1.0 × 1.0** and the grid to **n = 201** points per axis,
- sweeps PML **thickness** `npml` and **strength** via `eta` where `strength = eta * omega`,
- plots **Re/Im** (not modulus), plots **PML strength** `|sigma|`, and shows stretch factors on the **Re/Im axis**,
- computes a simple **reflection score** and visualizes a heatmap to pick a good `npml` range.

**About 4th/6th/8th order Laplacian:** your current `operators/assemble.py` uses a 2nd-order stencil.
So the PML tuning is fully project-native, and the FD-order comparison at the end is an **independent reference**
(to show what higher-order changes qualitatively).

## 0) Imports and path setup

Run from the project root (the folder that contains `src/`).

In [ ]:

from __future__ import annotations

import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 120

# --- make src importable ---
PROJECT_ROOT = Path.cwd()
SRC_ROOT = PROJECT_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))

# --- project imports ---
from core.config import Grid2D, HelmholtzConfig, PMLConfig
from core.cases import make_default_cases
from core.medium import build_medium
from core.rhs import assemble_rhs

from operators.pml import build_pml_profiles
from operators.solve import solve_helmholtz

print("Imports OK. SRC_ROOT =", SRC_ROOT)

## 1) Fixed physical domain and grid: n = 201 on 1.0 × 1.0

In [ ]:

Lx = 1.0
Ly = 1.0
n = 201

grid = Grid2D(nx=n, ny=n, lx=Lx, ly=Ly)
print("grid:", grid)
print("hx, hy:", grid.hx, grid.hy)

## 2) Choose a case + build `c(x,y)` and RHS `f`

In [ ]:

cases = make_default_cases()
print("Available cases:", list(cases.keys()))

In [ ]:

CASE_NAME = "constant" if "constant" in cases else list(cases.keys())[0]
case = cases[CASE_NAME]
print("Using case:", CASE_NAME, case)

In [ ]:

omega = 80.0  # change/sweep later if you want

cfg0 = HelmholtzConfig(omega=omega, grid=grid, pml=None)
X, Y = cfg0.grid.mesh()

c = build_medium(cfg0, case, X, Y)     # (nx, ny)
f = assemble_rhs(cfg0, case, X, Y)     # (N,) or (nx, ny)

print("c.shape:", c.shape, "c.min/max:", float(np.min(c)), float(np.max(c)))
print("f.shape:", f.shape, "dtype:", f.dtype)

## 3) Diagnostics helpers: Re/Im plots, sigma plots, reflection score

In [ ]:

def plot_re_im(U: np.ndarray, title: str):
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    im0 = ax[0].imshow(U.real, origin="lower")
    ax[0].set_title(f"Re(u) — {title}")
    plt.colorbar(im0, ax=ax[0], fraction=0.046)

    im1 = ax[1].imshow(U.imag, origin="lower")
    ax[1].set_title(f"Im(u) — {title}")
    plt.colorbar(im1, ax=ax[1], fraction=0.046)
    plt.show()

def sigma_strength_field(sig_x: np.ndarray, sig_y: np.ndarray) -> np.ndarray:
    Sx = sig_x[None, :].repeat(sig_y.size, axis=0)
    Sy = sig_y[:, None].repeat(sig_x.size, axis=1)
    return np.sqrt(Sx**2 + Sy**2)

def plot_sigma(sig_x: np.ndarray, sig_y: np.ndarray, title: str):
    Smag = sigma_strength_field(sig_x, sig_y)
    plt.figure(figsize=(5,4))
    plt.imshow(Smag, origin="lower")
    plt.title(title + " — |sigma(x,y)|")
    plt.colorbar(fraction=0.046)
    plt.show()

def plot_stretch_re_im(sx: np.ndarray, sy: np.ndarray, title: str):
    plt.figure(figsize=(5,4))
    plt.scatter(sx.real, sx.imag, s=8, alpha=0.7, label="s_x")
    plt.scatter(sy.real, sy.imag, s=8, alpha=0.7, label="s_y")
    plt.axhline(0, linewidth=1)
    plt.axvline(0, linewidth=1)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.title(title + " — stretch factors (Re/Im)")
    plt.xlabel("Re(s)")
    plt.ylabel("Im(s)")
    plt.legend()
    plt.show()

def reflection_score(U: np.ndarray, npml: int, ring: int = 8) -> float:
    # core = grid with npml stripped off each side
    core = U[npml:-npml, npml:-npml] if npml > 0 else U
    ny, nx = core.shape
    if 2*ring >= min(nx, ny):
        return np.nan

    mask = np.zeros_like(core, dtype=bool)
    mask[:ring, :] = True
    mask[-ring:, :] = True
    mask[:, :ring] = True
    mask[:, -ring:] = True

    E_ring = float(np.sum(np.abs(core[mask])**2))
    E_in = float(np.sum(np.abs(core[~mask])**2) + 1e-30)
    return E_ring / E_in

## 4) One PML run (m fixed to 2)

Your parameterization:
- `PMLConfig(thickness=npml, power=m, strength=strength)`
- with `strength = eta * omega`.

We fix `m = 2` here.

In [ ]:

npml = 20
eta = 1.5
m = 2

strength = float(eta) * float(omega)

cfg = HelmholtzConfig(
    omega=float(omega),
    grid=grid,
    pml=PMLConfig(thickness=int(npml), strength=float(strength), power=float(m)),
)

sol = solve_helmholtz(cfg, c=c, f=f, return_matrix=False, return_residual=True)
U = sol["U"]

print("Residual norms:", sol.get("norms", {}))
print("Reflection score:", reflection_score(U, npml=npml, ring=8))

In [ ]:

sig_x, sig_y, sx, sy = build_pml_profiles(cfg, c_ref=float(np.min(c)))

plot_sigma(sig_x, sig_y, title=f"PML strength (npml={npml}, eta={eta}, omega={omega}, m={m})")
plot_stretch_re_im(sx, sy, title="PML stretch factors")

plot_re_im(U, title="full grid (includes PML band)")
plot_re_im(U[npml:-npml, npml:-npml], title=f"core only (removed {npml} points each side)")

## 5) Sweep PML thickness × eta (strength)

Produces a heatmap of reflection score. Lower is better.

In [ ]:

def run_pml_score(npml: int, eta: float, omega: float) -> float:
    strength = float(eta) * float(omega)
    cfg = HelmholtzConfig(
        omega=float(omega),
        grid=grid,
        pml=PMLConfig(thickness=int(npml), strength=float(strength), power=2.0),
    )
    sol = solve_helmholtz(cfg, c=c, f=f, return_matrix=False, return_residual=False)
    return reflection_score(sol["U"], npml=npml, ring=8)

npml_list = [10, 15, 20, 25, 30]
eta_list  = [0.5, 1.0, 1.5, 2.0, 3.0]

scores = np.zeros((len(npml_list), len(eta_list)), dtype=float)
for i, a in enumerate(npml_list):
    for j, e in enumerate(eta_list):
        s = run_pml_score(npml=a, eta=e, omega=omega)
        scores[i, j] = s
        print(f"npml={a:>3d}, eta={e:>4.1f} -> score={s:.3e}")

In [ ]:

plt.figure(figsize=(7, 4))
plt.imshow(scores, origin="lower", aspect="auto")
plt.xticks(range(len(eta_list)), [str(e) for e in eta_list])
plt.yticks(range(len(npml_list)), [str(v) for v in npml_list])
plt.xlabel("eta (strength = eta * omega)")
plt.ylabel("npml (grid points)")
plt.title(f"PML tuning heatmap (omega={omega}) — lower is better")
plt.colorbar(fraction=0.046)
plt.show()

best = np.unravel_index(np.nanargmin(scores), scores.shape)
best_npml = npml_list[best[0]]
best_eta  = eta_list[best[1]]
print(f"Best (in this sweep): npml={best_npml}, eta={best_eta}, score={scores[best]:.3e}")

lam_min = 2*np.pi*float(np.min(c))/float(omega)
for npml_ in npml_list:
    thick = npml_ * float(max(grid.hx, grid.hy))
    print(f"npml={npml_:>3d} -> thickness={thick:.4f} (~{thick/lam_min:.2f} λ_min)")

## 6) Lock-in check: visualize the best candidate

In [ ]:

npml = int(best_npml)
eta = float(best_eta)
strength = eta * omega

cfg_best = HelmholtzConfig(
    omega=float(omega),
    grid=grid,
    pml=PMLConfig(thickness=npml, strength=float(strength), power=2.0),
)

sol = solve_helmholtz(cfg_best, c=c, f=f, return_matrix=False, return_residual=True)
U = sol["U"]
print("Residual norms:", sol.get("norms", {}))
print("Reflection score:", reflection_score(U, npml=npml, ring=8))

sig_x, sig_y, sx, sy = build_pml_profiles(cfg_best, c_ref=float(np.min(c)))

plot_sigma(sig_x, sig_y, title=f"BEST PML strength (npml={npml}, eta={eta}, omega={omega}, m=2)")
plot_stretch_re_im(sx, sy, title="BEST stretch factors")
plot_re_im(U, title="BEST full grid")
plot_re_im(U[npml:-npml, npml:-npml], title="BEST core only")

## 7) Reference: 4th vs 6th vs 8th order Laplacian (independent)

Your current project assembler does not implement higher-order stencils.
This section shows what changes qualitatively when you do use 4th/6th/8th order stencils.

In [ ]:

import scipy.sparse as sp
import scipy.sparse.linalg as spla

def d2_stencil(order: int):
    if order == 4:
        offs = np.array([-2, -1, 0, 1, 2])
        coefs = np.array([-1, 16, -30, 16, -1]) / 12.0
    elif order == 6:
        offs = np.array([-3, -2, -1, 0, 1, 2, 3])
        coefs = np.array([2, -27, 270, -490, 270, -27, 2]) / 180.0
    elif order == 8:
        offs = np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])
        coefs = np.array([-9, 128, -1008, 8064, -14350, 8064, -1008, 128, -9]) / 5040.0
    else:
        raise ValueError("order must be 4, 6, or 8")
    return offs, coefs

def make_1d_d2(n: int, h: float, order: int) -> sp.csr_matrix:
    offs, coefs = d2_stencil(order)
    diags = []
    for off, c_ in zip(offs, coefs):
        diags.append(c_ * np.ones(n - abs(off)))
    D2 = sp.diags(diags, offs, shape=(n, n), format="csr") / (h**2)

    # Dirichlet boundary rows
    for k in [0, n-1]:
        D2[k, :] = 0
        D2[k, k] = 1.0
    return D2

def gaussian_rhs_2d(n: int, h: float, x0=0.5, y0=0.5, width_cells=2.5):
    xs = np.linspace(0, 1.0, n)
    ys = np.linspace(0, 1.0, n)
    X, Y = np.meshgrid(xs, ys, indexing="ij")
    sig = width_cells * h
    f = np.exp(-((X-x0)**2 + (Y-y0)**2)/(2*sig**2))
    return f.astype(np.complex128)

def solve_const_order(n: int, h: float, omega: float, order: int):
    Dxx = make_1d_d2(n, h, order)
    Dyy = make_1d_d2(n, h, order)
    I = sp.eye(n, format="csr")
    Lap = sp.kron(I, Dxx) + sp.kron(Dyy, I)
    A = -Lap - (omega**2) * sp.eye(n*n, format="csr")  # matches (-Δ - k^2)
    f = gaussian_rhs_2d(n, h).reshape(-1)
    u = spla.spsolve(A, f).reshape(n, n)
    return u

for order in [4, 6, 8]:
    Uo = solve_const_order(n=n, h=float(grid.hx), omega=float(omega), order=order)
    plot_re_im(Uo, title=f"REFERENCE constant-coefficient: order={order}")